In [0]:
from google.colab import drive
import os
import torch
import torch.nn as nn
import torchvision.transforms as T
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [7]:
print(f"CUDA is available? {torch.cuda.is_available()}")
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(dev)

CUDA is available? True
cuda


In [8]:
drive.mount("/content/drive",True)
root_dir = "/content/drive/My Drive/SB3/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
train_transform = T.Compose([
    T.Resize(256),
    T.ToTensor()
])

In [0]:
train_dataset = ImageFolder(os.path.join(root_dir, "train"), transform=train_transform)
test_dataset = ImageFolder(os.path.join(root_dir, "test"), transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size = 4, num_workers=4, shuffle=True)
test_loader   = DataLoader(test_dataset,   batch_size = 4, num_workers=4, shuffle=False)
loaders = {
    "train": train_loader,
    "test": test_loader
}

In [78]:
num_classes = len(train_dataset.classes)
print(num_classes)

8


In [0]:
class ConvCapsLayer(nn.Module):
    def __init__(self, in_channel_types, out_channel_types, in_vector_length, out_vector_length, kernel_size, stride, num_iterations, new_height, new_width):
      # Call parent constructor
      super().__init__()
      
      self.in_channel_types = in_channel_types
      self.out_channel_types = out_channel_types
      self.in_vector_length = in_vector_length
      self.out_vector_length = out_vector_length
      self.kernel_size = kernel_size
      self.stride = stride
      self.num_iterations = num_iterations
      self.new_height = new_height
      self.new_width = new_width
      self.W = nn.Parameter(torch.randn(1, in_channel_types, 1, out_channel_types, out_vector_length, in_vector_length))
    
    def squash(self, in_tensor):
      squared_norm = (in_tensor ** 2).sum(-2, keepdim=True)
      out_tensor = squared_norm *  in_tensor / ((1. + squared_norm) * torch.sqrt(squared_norm))
      return out_tensor

    def forward(self, x):
      batch_size = x.size(0)
      y = torch.zeros(batch_size, self.in_vector_length, self.in_channel_types, self.new_height, self.new_width, dtype=torch.float).to(dev)
      kernel = torch.ones(1, 1, self.kernel_size, self.kernel_size, dtype=torch.float).to(dev)
      for i in range(self.in_vector_length):
        for j in range(self.in_channel_types):
          y[:,i,j,:,:] = F.conv2d(x[:,i,j,:,:].unsqueeze_(1), kernel, stride=self.stride).squeeze(1)

      y = y.view(batch_size, self.in_channel_types, self.new_height * self.new_width, -1)
      y = torch.stack([y] * self.out_channel_types, dim=3).unsqueeze(5)

      W = torch.cat([self.W] * batch_size, dim=0)
      c = W @ y

      b_ij = torch.zeros(1, self.in_channel_types, c.size(2), self.out_channel_types, 1).to(dev)

      for iteration in range(self.num_iterations):
        c_ij = F.softmax(b_ij, dim=-2)
        c_ij = torch.cat([c_ij] * batch_size, dim=0).unsqueeze(5)

        s_j = (c_ij * c).sum(dim=1, keepdim=True)
        v_j = self.squash(s_j)

        if iteration < self.num_iterations - 1:
          a_ij = torch.matmul(c.transpose(4, 5), torch.cat([v_j] * self.in_channel_types, dim=1))
          b_ij = b_ij + a_ij.squeeze(5).mean(dim=0, keepdim=True)

      v_j = v_j.squeeze(1)
      v_j = v_j.transpose(3,1).squeeze(4)
      v_j = v_j.view(v_j.size(0), v_j.size(1), v_j.size(2), self.new_height, self.new_width)
      return v_j

In [0]:
class CapsNet(nn.Module):
  # Constructor
  def __init__(self):
    # Call parent constructor
    super().__init__()
    # Create convolutional layers
    self.conv_layers = nn.Sequential(
      # Layer 1
      nn.Conv2d(3, 16, kernel_size=5, padding=2, stride=2),
      nn.ReLU()
    )
    self.conv_capsule_layers = nn.Sequential(
      # Layer 1
      ConvCapsLayer(in_channel_types = 1, out_channel_types = 2, in_vector_length = 16, out_vector_length = 16, kernel_size = 2, stride = 2, num_iterations = 2, new_height = 64, new_width = 64)
    )
  
  # Forward
  def forward(self, x):
    x = self.conv_layers(x)
    x = x.unsqueeze(2)
    x = self.conv_capsule_layers(x)
    return x

In [133]:
batch,labels = next(iter(train_loader))
batch = batch.to(dev)
labels = labels.to(dev)
model = CapsNet()
model = model.to(dev)
out = model(batch)
print(out.size())

torch.Size([4, 16, 2, 64, 64])
